#### Import necessary packages

In [1]:
import boto3
import pandas as pd
#import mysql.connector
from collections import OrderedDict
from datetime import datetime, timedelta

#### Connect to S3 and download CSV files as Pandas Dataframe

In [2]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    resp = s3.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [3]:
s3 = boto3.client('s3')

In [4]:
bucket = 'cms-data-enrollment'

In [5]:
files = get_s3_keys(bucket)

In [6]:
idx = -1
files[idx]

'CPSC_Enrollment_Info_2020_01.csv'

In [7]:
obj = s3.get_object(Bucket= bucket, Key= files[idx])

In [8]:
latest_df = pd.read_csv(obj['Body']) # 'Body' is a key word

####  Create initial dataframe to which we will merge all other enrollment data

Create a list of dates to be used as column values in our dataframes

In [9]:
dates = ["2016-01-01", "2020-01-02"]
start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
file_months = list(OrderedDict(((start + timedelta(_)).strftime(r"%b-%y"), None) for _ in range((end - start).days)).keys())

1.A Create "Unique_ID" column by combining "Contract Number" and "Plan ID" which will be used to join the master contract data in order to match the parent organization to each plan/enrollment data

In [10]:
unique_id = latest_df["Contract Number"].map(str) + '-' + latest_df["Plan ID"].map(str)
latest_df.insert(loc=0, column='Unique_ID', value=unique_id)

1.B. Create another "Unique_ID_SSA" column which will be used for joining other enrollment data

In [11]:
unique_id_v1 = latest_df["Unique_ID"].map(str) + '-' + latest_df["SSA State County Code"].map(str)
latest_df.insert(loc=0, column='Unique_ID_SSA', value=unique_id_v1)
latest_df.head()

,Unique_ID_SSA,Unique_ID,Contract Number,Plan ID,SSA State County Code,FIPS State County Code,State,County,Enrollment
0,E0654-801-2275,E0654-801,E0654,801,2275,NaN,NaN,NaN,*
1,E0654-801-2198,E0654-801,E0654,801,2198,NaN,NaN,NaN,*
2,E0654-801-2195,E0654-801,E0654,801,2195,NaN,NaN,NaN,*
3,E0654-801-1000,E0654-801,E0654,801,1000,1001.0,AL,Autauga,*
4,E0654-801-1010,E0654-801,E0654,801,1010,1003.0,AL,Baldwin,12


2. Replace all "*" values with the integer 5

In [12]:
latest_df['Enrollment'] = latest_df['Enrollment'].str.replace('*','5').apply(pd.to_numeric)

3. Drop all the rows that have NaN values (only a couple rows do not have State/County codes

In [13]:
latest_df.dropna(inplace=True)

4. Change the 'Enrollment' column header to the month/date

In [14]:
# latest_df['Date'] = datetime.strptime(file_months[idx], "%b-%y")
latest_df.rename(columns={'Enrollment': file_months[idx]}, inplace=True)

#### Join the Provider/Plan from the Contract_info file

##### Import the master plan/state data file

In [15]:
contract_df = pd.read_csv('clean_cms_contract_data.csv')

In [16]:
contract_df = contract_df[['Unique_ID', 'Parent Organization']]

##### Merge the last month's enrollment data file with the master contract file on the 'Unique_ID"

In [17]:
master_data = pd.merge(latest_df, contract_df, left_on = 'Unique_ID', right_on = 'Unique_ID', how='right')

In [18]:
master_data.drop(['Contract Number', 'Plan ID', 'SSA State County Code', 'Unique_ID'], axis=1, inplace=True)

In [19]:
master_data.dropna(inplace = True)

In [20]:
master_data.head()

,Unique_ID_SSA,FIPS State County Code,State,County,Jan-20,Parent Organization
0,E0654-801-1000,1001.0,AL,Autauga,5.0,IBT Voluntary Employee Benefits Trust
1,E0654-801-1010,1003.0,AL,Baldwin,12.0,IBT Voluntary Employee Benefits Trust
2,E0654-801-1020,1005.0,AL,Barbour,5.0,IBT Voluntary Employee Benefits Trust
3,E0654-801-1030,1007.0,AL,Bibb,5.0,IBT Voluntary Employee Benefits Trust
4,E0654-801-1040,1009.0,AL,Blount,5.0,IBT Voluntary Employee Benefits Trust


#### Now add each additional month to the master data by using 'left' joins to master_data

1. Import data object and read the csv file

In [22]:
n = 1
obj = s3.get_object(Bucket= bucket, Key= files[idx-n])

In [23]:
new_enrollments = pd.read_csv(obj['Body']) # 'Body' is a key word

2. Add 'unique_id_ssa' which will be the column that we join on

In [24]:
unique_id_ssa = new_enrollments["Contract Number"].map(str) + '-' + new_enrollments["Plan ID"].map(str) + '-' + new_enrollments["SSA State County Code"].map(str)
new_enrollments.insert(loc=0, column='Unique_ID_SSA', value=unique_id_ssa)

In [25]:
new_enrollments['Enrollment'] = new_enrollments['Enrollment'].str.replace('*','5').apply(pd.to_numeric)

3. Change the 'Enrollment' column header to the year/month for the enrollment file

In [26]:
new_enrollments.rename(columns={'Enrollment': file_months[idx-n]}, inplace=True)

In [27]:
new_enrollments.dropna(inplace=True)

4. Drop unneccessary columns which will clog up memory

In [28]:
new_enrollments.drop(['Contract Number', 'Plan ID', 'SSA State County Code',
       'FIPS State County Code', 'State', 'County'], axis=1, inplace=True)

In [29]:
print(file_months[idx-n])
new_enrollments.head()

Dec-19


,Unique_ID_SSA,Dec-19
3,E0654-801-1000,5
4,E0654-801-1010,5
5,E0654-801-1020,5
6,E0654-801-1030,5
7,E0654-801-1040,5


##### Merge the enrollment data and new enrollments 

In [30]:
# If this is the first 'merge' of the master df with enrollments, then we run this code below
updated_df = pd.merge(master_data, new_enrollments, left_on = 'Unique_ID_SSA', right_on = 'Unique_ID_SSA', how='left')

# If we are updating the dataframe with additional enrollment, then we merge 'updated_df' with new 'enrollments' files
# updated_df = pd.merge(updated_df, new_enrollments, left_on = 'Unique_ID_SSA', right_on = 'Unique_ID_SSA', how='left')

In [31]:
updated_df.head()

,Unique_ID_SSA,FIPS State County Code,State,County,Jan-20,Parent Organization,Dec-19
0,E0654-801-1000,1001.0,AL,Autauga,5.0,IBT Voluntary Employee Benefits Trust,5.0
1,E0654-801-1010,1003.0,AL,Baldwin,12.0,IBT Voluntary Employee Benefits Trust,5.0
2,E0654-801-1020,1005.0,AL,Barbour,5.0,IBT Voluntary Employee Benefits Trust,5.0
3,E0654-801-1030,1007.0,AL,Bibb,5.0,IBT Voluntary Employee Benefits Trust,5.0
4,E0654-801-1040,1009.0,AL,Blount,5.0,IBT Voluntary Employee Benefits Trust,5.0


In [33]:
updated_df.fillna(0 ,inplace=True)

##### Group the plans to the Parent Organization

In [ ]:
# This is a test run
updated_df.groupby(['State', 'FIPS State County Code', 'Parent Organization'], as_index=False).sum()

##### Enrollment data both grouped and summarized at the county level

In [34]:
county_data = updated_df.groupby(['State', 'FIPS State County Code', 'Parent Organization'], as_index=False).sum()

In [35]:
county_data.to_csv('county_payer_stats.csv')

#### Connect to MySql and create database

In [ ]:
con = mysql.connector.connect(user='admin', 
                              password='padatascience',
                              host='cms-data.clojwkv1g9c0.us-east-2.rds.amazonaws.com')

In [ ]:
cursor = con.cursor()

In [ ]:
cursor.execute("CREATE DATABASE IF NOT EXISTS CMS_Table")

#### Use sqlalchemy to insert Pandas dataframe as table to database

In [ ]:
database_username = 'admin'
database_password = 'padatascience'
database_ip       = 'cms-data.clojwkv1g9c0.us-east-2.rds.amazonaws.com'
database_name     = 'CMS_Table'

In [ ]:
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [ ]:
enrollment_data.to_sql(con=database_connection, name='CMS-Data', if_exists='append', chunksize=10000)

In [ ]:
cursor.close() 

#### Put it all together and export to MySQL database

In [ ]:
i = 0
rows = 0
for file in files:
    print(file)
    obj = s3.get_object(Bucket= bucket, Key= file)
    raw_df = pd.read_csv(obj['Body'])
    # ETL Step 1
    unique_id = raw_df["Contract Number"].map(str) + '-' + raw_df["Plan ID"].map(str)
    raw_df.insert(loc=0, column='Unique_ID', value=unique_id)
    # ETL Step 2
    print('Applying numerics')
    raw_df['Enrollment'] = raw_df['Enrollment'].str.replace('*','5').apply(pd.to_numeric)
    # ETL Step 3
    raw_df.dropna(inplace=True)
    # ETL Step 4
    raw_df['Date'] = datetime.strptime(file_months[i], "%b-%y")
    i += 1
    # ETL Step 5
    enrollment_data = pd.merge(raw_df, contract_df[['Unique_ID', 'Plan Name']], left_on = 'Unique_ID', right_on = 'Unique_ID', how='left')
    # Count the number of rows in dataframe
    print('Number of rows: ', enrollment_data.shape[0], ' i = ', i)
    rows += enrollment_data.shape[0]
    # SQL Load
    enrollment_data.to_sql(con=database_connection, name='CMS-Data', if_exists='append', chunksize=10000)
    print('total number of rows uploaded: ', rows)
print('Now we are all finished!')
print('The number of rows: ', rows)

In [ ]:
len(file_months)

In [ ]:
i